In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from training.trainer import *
import optuna
from tqdm.notebook import tqdm

In [2]:
def read_data(cluster_num):
    df_train = pd.read_parquet(f'./clean_data/train/cluster_{cluster_num}.gz')
    df_test = pd.read_parquet(f'./clean_data/test/cluster_{cluster_num}.gz')
    df_attr = pd.read_csv('./clean_data/attr.csv')
    # soc-dem features u
    df_train = df_train.merge(df_attr, 
                              how = 'left', 
                              left_on = ['ego_id', 'u'],
                              right_on = ['ego_id', 'u']).rename(
                                                                columns = {
                                                                    'school':     'school_u',
                                                                    'university': 'university_u',
                                                                    'sex':        'sex_u',
                                                                    'city_id':    'city_id_u',
                                                                    'age':        'age_u'
                                                                    }
                                                                )
    df_test = df_test.merge(df_attr, 
                              how = 'left', 
                              left_on = ['ego_id', 'u'],
                              right_on = ['ego_id', 'u']).rename(
                                                                columns = {
                                                                    'school':     'school_u',
                                                                    'university': 'university_u',
                                                                    'sex':        'sex_u',
                                                                    'city_id':    'city_id_u',
                                                                    'age':        'age_u'
                                                                    }
                                                                )
    # soc-dem features v
    df_train = df_train.merge(df_attr, 
                              how = 'left', 
                              left_on = ['ego_id', 'v'],
                              right_on = ['ego_id', 'u']).drop(['u_y'], axis = 1).rename(
                                                                columns = {
                                                                    'school':     'school_v',
                                                                    'university': 'university_v',
                                                                    'sex':        'sex_v',
                                                                    'city_id':    'city_id_v',
                                                                    'age':        'age_v',
                                                                    'u_x': 'u'
                                                                    }
                                                                )
    df_test = df_test.merge(df_attr, 
                              how = 'left', 
                              left_on = ['ego_id', 'v'],
                              right_on = ['ego_id', 'u']).drop(['u_y'], axis = 1).rename(
                                                                columns = {
                                                                    'school':     'school_v',
                                                                    'university': 'university_v',
                                                                    'sex':        'sex_v',
                                                                    'city_id':    'city_id_v',
                                                                    'age':        'age_v',
                                                                    'u_x': 'u'
                                                                    }
                                                                )
    return df_train, df_test, df_attr

In [3]:
def preproc_and_get_features(df, df_attr):
    for i in ['city_id_u', 'city_id_v', 'sex_u', 'sex_v', 'school_u', 'university_u', 'university_v', 'school_v', 't']:
        df[i].fillna(-1, inplace = True)
    iqr = np.quantile(df_attr['age'], 0.75)-np.quantile(df_attr['age'], .25)
    m_age = df_attr[~(df_attr['age'].isna())]['age'].median()
    df['age_u'].fillna(m_age, inplace = True)
    df['age_v'].fillna(m_age, inplace = True)
    df.loc[df['age_u']>m_age+1.5*iqr, 'age_u'] = m_age
    df.loc[df['age_v']>m_age+1.5*iqr, 'age_v'] = m_age

    df.loc[(df['university_u'] == df['university_v']) & (df['university_u'] != -1), 'university_is_equal'] = 1
    df.loc[(df['university_u'] == -1) | (df['university_v'] == -1), 'university_is_equal'] = -1
    df['university_is_equal'].fillna(0, inplace = True)
    
    df.loc[(df['school_u'] == df['school_v']) & (df['school_u'] != -1), 'school_is_equal'] = 1
    df.loc[(df['school_u'] == -1) | (df['school_v'] == -1), 'school_is_equal'] = -1
    df['school_is_equal'].fillna(0, inplace = True)
    
    df.loc[(df['city_id_u'] == df['city_id_v']) & (df['city_id_u'] != -1), 'city_is_equal'] = 1
    df.loc[(df['city_id_v'] == -1) | (df['city_id_u'] == -1), 'city_is_equal'] = -1
    df['city_is_equal'].fillna(0, inplace = True)
    
    df.loc[(df['sex_u'] == df['sex_v']) & (df['sex_v'] != -1), 'sex_is_equal'] = 1
    df.loc[(df['sex_u'] == -1) | (df['sex_v'] == -1), 'sex_is_equal'] = -1
    df['sex_is_equal'].fillna(0, inplace = True)
    
    df['age_diff'] = df['age_u']-df['age_v']
    df.loc[abs(df['age_diff']) < 1, 'age_diff_1'] = 1
    df.loc[~(abs(df['age_diff']) < 1), 'age_diff_1'] = 0
    
    df.loc[abs(df['age_diff']) < 2, 'age_diff_2'] = 1
    df.loc[~(abs(df['age_diff']) < 2), 'age_diff_2'] = 0
    
    df.loc[abs(df['age_diff']) < 3, 'age_diff_3'] = 1
    df.loc[~(abs(df['age_diff']) < 3), 'age_diff_3'] = 0
    
    df.loc[abs(df['age_diff']) < 4, 'age_diff_4'] = 1
    df.loc[~(abs(df['age_diff']) < 4), 'age_diff_4'] = 0
    
    df.loc[abs(df['age_diff']) < 5, 'age_diff_5'] = 1
    df.loc[~(abs(df['age_diff']) < 5), 'age_diff_5'] = 0
    
    df.loc[abs(df['age_diff']) < 10, 'age_diff_10'] = 1
    df.loc[~(abs(df['age_diff']) < 10), 'age_diff_10'] = 0
    
    df.loc[abs(df['age_diff']) < 15, 'age_diff_15'] = 1
    df.loc[~(abs(df['age_diff']) < 15), 'age_diff_15'] = 0
    
    df.loc[abs(df['age_diff']) < 20, 'age_diff_20'] = 1
    df.loc[~(abs(df['age_diff']) < 20), 'age_diff_20'] = 0
    
    df.loc[abs(df['age_diff']) < 25, 'age_diff_25'] = 1
    df.loc[~(abs(df['age_diff']) < 25), 'age_diff_25'] = 0
    
    df.loc[abs(df['age_diff']) < 30, 'age_diff_30'] = 1
    df.loc[~(abs(df['age_diff']) < 30), 'age_diff_30'] = 0
    
    df.loc[(df['school_is_equal'] == 1) & (df['age_diff_2'] == 1), 'flg_classmates'] = 1
    df.loc[~((df['school_is_equal'] == 1) & (df['age_diff_2'] == 1)), 'flg_classmates'] = 0
    
    df.loc[(df['university_is_equal'] == 1) & (df['age_diff_2'] == 1), 'flg_univermates'] = 1
    df.loc[~((df['university_is_equal'] == 1) & (df['age_diff_2'] == 1)), 'flg_univermates'] = 0
    
    df.loc[df['u']==0, 'from_ego'] = 1
    df.loc[~(df['u']==0), 'from_ego'] = 1
    df.loc[df['v']==0, 'to_ego'] = 1
    df.loc[~(df['v']==0), 'to_ego'] = 1

    df.drop(['university_u', 'university_v', 'city_id_u', 'city_id_v', 'school_u', 'school_v'], axis = 1, inplace = True)
    return df

In [4]:
tech_cols = ['ego_id', 'u', 'v', 'x1']

In [5]:
def train_model(train):
    x = train.drop(tech_cols, axis = 1)
    y = train['x1']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.8, 
                                                        shuffle = True, random_state = 42)
    best_params = calc_hps('LGBM', x_train, y_train, 'reg', trials_num = 5)
    model = lgb.LGBMRegressor()
    model.set_params(**best_params)
    model.fit(x_train, y_train)

    rmse_train = mean_squared_error(y_train, model.predict(x_train), squared = False)
    rmse_test = mean_squared_error(y_test, model.predict(x_test), squared = False)
    
    return model, rmse_train, rmse_test

In [6]:
def make_submition():
    df_subm = pd.read_csv('./clean_data/submission.csv')
    test_pred = pd.DataFrame(columns = ['ego_id', 'u', 'v', 'pred'])
    for cluster in tqdm(range(9)):
        _, test, attr = read_data(cluster)
        test = preproc_and_get_features(test, attr)
        model = lgb.Booster(model_file=f'./models/cluster_{cluster}_model.txt')
        test['pred'] = model.predict(test[model.feature_name()])
        test_pred = pd.concat([test_pred, test[['ego_id', 'u', 'v', 'pred']]], axis = 0)
    df_subm = df_subm.merge(test_pred, how = 'left', left_on = ['ego_id', 'u', 'v'],
                           right_on = ['ego_id', 'u', 'v']).drop('x1', axis = 1)\
                .rename(columns = {'pred': 'x1'}).drop_duplicates(['ego_id','u', 'v'], keep='first')[['ego_id', 'u','v', 'x1']]
    df_subm.to_csv('./submitions/submition.csv', index = False)
    return df_subm

In [7]:
models = {}
for cluster in tqdm(range(9)):
    train, test, attr = read_data(cluster)
    train = preproc_and_get_features(train, attr)
    test = preproc_and_get_features(test, attr)
    del attr
    model, rmse_train, rmse_val = train_model(train)
    models[cluster] = model
    model.booster_.save_model(f'./models/cluster_{cluster}_model.txt')
    
    print(f'----- Cluster {cluster} -----')
    print(f'RMSE TRAIN: {rmse_train:.3f}')
    print(f'RMSE VAL: {rmse_val:.3f}')

  0%|          | 0/9 [00:00<?, ?it/s]

[I 2023-05-21 04:56:38,456] A new study created in memory with name: no-name-3d0da58c-ff66-4d57-9de2-01edc7809bec
[I 2023-05-21 04:56:46,433] Trial 0 finished with value: -0.6038514067729256 and parameters: {'max_depth': 10, 'num_leaves': 98, 'n_estimators': 147, 'reg_alpha': 0.4181207578712726, 'reg_lambda': 2.6913857515375475e-07, 'min_child_samples': 37, 'min_child_weight': 1.9597272946419337}. Best is trial 0 with value: -0.6038514067729256.
[I 2023-05-21 04:56:49,580] Trial 1 finished with value: -0.696370783023756 and parameters: {'max_depth': 4, 'num_leaves': 28, 'n_estimators': 69, 'reg_alpha': 0.029837007071720243, 'reg_lambda': 1.8631412772492373, 'min_child_samples': 26, 'min_child_weight': 9.712061172465253}. Best is trial 0 with value: -0.6038514067729256.
[I 2023-05-21 04:56:53,521] Trial 2 finished with value: -0.700024309639072 and parameters: {'max_depth': 3, 'num_leaves': 64, 'n_estimators': 129, 'reg_alpha': 0.0032865051905924905, 'reg_lambda': 9.176850523806595e-05,

----- Cluster 0 -----
RMSE TRAIN: 0.604
RMSE VAL: 0.657


[I 2023-05-21 04:57:50,558] A new study created in memory with name: no-name-261c504d-52fa-4413-b8f7-ded3110b616d
[I 2023-05-21 04:57:53,608] Trial 0 finished with value: -0.6248107670668874 and parameters: {'max_depth': 9, 'num_leaves': 82, 'n_estimators': 111, 'reg_alpha': 0.004983254087499036, 'reg_lambda': 0.004028405287950032, 'min_child_samples': 46, 'min_child_weight': 2.6613185072808148}. Best is trial 0 with value: -0.6248107670668874.
[I 2023-05-21 04:57:57,076] Trial 1 finished with value: -0.623971076996371 and parameters: {'max_depth': 8, 'num_leaves': 92, 'n_estimators': 140, 'reg_alpha': 0.09196685384261512, 'reg_lambda': 1.1880037706600035, 'min_child_samples': 57, 'min_child_weight': 0.8884351732088747}. Best is trial 1 with value: -0.623971076996371.
[I 2023-05-21 04:57:58,841] Trial 2 finished with value: -0.7038454737355568 and parameters: {'max_depth': 4, 'num_leaves': 20, 'n_estimators': 107, 'reg_alpha': 1.67994228552334e-06, 'reg_lambda': 3.4315186561583304, 'mi

----- Cluster 1 -----
RMSE TRAIN: 0.600
RMSE VAL: 0.685


...

----- Cluster 6 -----
RMSE TRAIN: 0.627
RMSE VAL: 0.646


[I 2023-05-21 05:12:43,717] A new study created in memory with name: no-name-f1abc3bf-e575-469f-9660-68ffa207ca35
[I 2023-05-21 05:13:02,487] Trial 0 finished with value: -0.6036868517234363 and parameters: {'max_depth': 10, 'num_leaves': 99, 'n_estimators': 109, 'reg_alpha': 0.013261049818641919, 'reg_lambda': 0.5796967997707857, 'min_child_samples': 15, 'min_child_weight': 9.492703860625026}. Best is trial 0 with value: -0.6036868517234363.
[I 2023-05-21 05:13:14,085] Trial 1 finished with value: -0.650901798317116 and parameters: {'max_depth': 4, 'num_leaves': 24, 'n_estimators': 136, 'reg_alpha': 3.130349097950584e-05, 'reg_lambda': 5.677670011742192e-06, 'min_child_samples': 11, 'min_child_weight': 6.549609550952463}. Best is trial 0 with value: -0.6036868517234363.
[I 2023-05-21 05:13:22,919] Trial 2 finished with value: -0.7080096578198266 and parameters: {'max_depth': 2, 'num_leaves': 26, 'n_estimators': 106, 'reg_alpha': 0.0664024871841602, 'reg_lambda': 9.483006055982327e-08,

----- Cluster 7 -----
RMSE TRAIN: 0.604
RMSE VAL: 0.627


[I 2023-05-21 05:15:17,623] A new study created in memory with name: no-name-b8d8f7e8-b831-4103-8e07-7a815ab3f9ae
[I 2023-05-21 05:15:20,110] Trial 0 finished with value: -0.5564562508621911 and parameters: {'max_depth': 9, 'num_leaves': 78, 'n_estimators': 130, 'reg_alpha': 0.13553984668324973, 'reg_lambda': 0.00040492869611053763, 'min_child_samples': 29, 'min_child_weight': 5.715421368431086}. Best is trial 0 with value: -0.5564562508621911.
[I 2023-05-21 05:15:21,500] Trial 1 finished with value: -0.6610517884044214 and parameters: {'max_depth': 4, 'num_leaves': 43, 'n_estimators': 124, 'reg_alpha': 0.016237629257358752, 'reg_lambda': 1.2346426267112125, 'min_child_samples': 40, 'min_child_weight': 6.24102470353816}. Best is trial 0 with value: -0.5564562508621911.
[I 2023-05-21 05:15:22,690] Trial 2 finished with value: -0.6899215095149245 and parameters: {'max_depth': 3, 'num_leaves': 69, 'n_estimators': 117, 'reg_alpha': 2.897184249740328, 'reg_lambda': 2.2535949762330154e-05, '

----- Cluster 8 -----
RMSE TRAIN: 0.556
RMSE VAL: 0.656


In [8]:
submition = make_submition()

  0%|          | 0/9 [00:00<?, ?it/s]